In [1]:
import json
import urllib2
import urllib
import xmltodict
import os
import string
from ebooklib import epub 
from  comFunc import *
from multiprocessing import Pool
import time
import re
import functools

In [2]:
def movet(titles):
    metakey='http://purl.org/dc/elements/1.1/'
    f=titles[1][0]+"/"+titles[0]+".epub"
    ti=titles[1][2]['title']
    tmt=''.join(tt for tt in ti if tt  in string.ascii_letters+string.digits+" ()")
    series=''
    index=-1
    if tmt.count('(')>0:
        title=tmt[:tmt.index('(')].strip()
        seri=tmt[tmt.index('(')+1:tmt.index(")")].split()
        try:
            index=float(seri[-1])
            
            series=' '.join(seri[:-1]).strip()
        except:
            series=' '.join(seri).strip()
    else:
        title=''.join(tt for tt in ti if tt  in string.ascii_letters+string.digits+" '-:")
    #print index 
    au=''.join (t for t in titles[1][2]['author'].strip() if t in string.ascii_letters+" ." )
    if len(au.split())<2:
        return "A",f
    auth=' '.join(au.split())
    title=title.replace ("By "+au,'').strip()
    title=' '.join(m.strip() for m in title.split())
    if title.count(':')>0:
        title=title.split(':')[0].strip()
    tt="/media/sf_lbuntu/books/"+au.strip()+" - "+title.strip()+".epub"
    tmp="/tmp/"+auth.strip()+" - "+title.strip()+".epub"
    try:
        book=epub.read_epub(f)
    except:
        return "R",f
    if 'identifier' in book.metadata[metakey].keys():
        book.metadata[metakey].pop('identifier')
        book.set_identifier('0')
    if 'publisher' in book.metadata[metakey].keys():
        book.metadata[metakey].pop('publisher')
    if 'date' in book.metadata[metakey].keys():
            book.metadata[metakey].pop('date')
    
    if not os.path.isfile(tt):
        try:
            epub.write_epub(tmp,book,{} )
        except:
            return "W",f,tmp
            

        if os.path.exists(tmp):

            !ebook-meta "$tmp" --title "$ti" --authors "$auth" --language "eng" >>/dev/null 2>>/dev/null

            if index>0 and len(series)>0: 
                !ebook-meta "$tmp" --series "$series" --index "$index" >>/dev/null 2>>/dev/null

            !mv "$tmp" "$tt"
            return "S",f
        else:
            return "F",f
    else:
            return "E",f
#E=Exists, F=failed, S=success, W=Writing error, R=reading error, A=Author problem

In [3]:
kv=['The Mystery of the Blue Train_ - Agatha Christie',titles['The Mystery of the Blue Train_ - Agatha Christie']]
movet(kv)

NameError: name 'titles' is not defined

In [13]:
titles=load_obj('data/titlesImNNN')#{}
#unpacktitles
uptit=[[k,titles[k]] for k in titles.keys()]
prockeys=[]#load_obj('data/prockeysIm')
flist=list(filter(lambda x: x[0] not in prockeys, uptit))

len(flist),len(titles.keys()),len(prockeys)

(4484, 4484, 0)

In [14]:
pbooks=load_obj('data/procbooksIm')
t0 = time.time()
p = Pool(10)
books=p.map(movet, flist)
#p.join()
p.terminate()
p.close()
for b in books:
    k=b[1][b[1].rindex('/')+1:-5]
    pbooks[k]=b[0]
    prockeys.append(k)
save_obj(prockeys,'data/prockeysIm')
save_obj(pbooks,'data/procbooksIm')
t1=time.time()
total = t1-t0
print('time: ',total)

('time: ', 151.9607970714569)


In [15]:
len(books),len(prockeys)

(4484, 4484)

In [22]:
print "R: ",len(list (filter(lambda x:x[0]  =="R",books)))
print "W: ",len(list (filter(lambda x:x[0]  =="W",books)))
print "S: ",len(list (filter(lambda x:x[0]  =="S",books)))
print "F: ",len(list (filter(lambda x:x[0]  =="F",books)))
print "E: ",len(list (filter(lambda x:x[0]  =="E",books)))
print "A: ",len(list (filter(lambda x:x[0]  =="A",books)))

R:  82
W:  270
S:  2
F:  0
E:  3895
A:  235
